In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json
from pandas.io.json import json_normalize

In [2]:
def get_team_soup(team):
    # grabs the url for a particular team and returns a Beautifulsoup object containing player data for a that team
    xg_url = 'https://understat.com/team/' + team
    
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    driver.get(xg_url)
    team_soup = BeautifulSoup(driver.page_source, 'lxml')
    
    return team_soup

In [3]:
def get_EPL_soup():
    # grabs the url for the EPL and returns a Beautifulsoup object for each team(not player data) in the league
    xg_url = 'https://understat.com/league/EPL'
    
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    driver.get(xg_url)
    EPL_soup = BeautifulSoup(driver.page_source, 'lxml')
    
    return EPL_soup

In [4]:
def xG_EPL_Scraper(soup):
    # Returns a dataframe containing team xg data for all teams in the EPL
    
    headers = soup.find('div', attrs={'class':'chemp margin-top jTable'}).find('table').find_all('th',attrs={'class':'sort'})
    headers_list = []
    
    for header in headers:
        headers_list.append(header.get_text(strip=True))
    
    body = soup.find('div', attrs={'class':'chemp margin-top jTable'}).table.tbody
    all_rows_list = []

    for tr in body.find_all('tr'):
        row = tr.find_all('td')
        current_row = []
        for item in row:
            current_row.append(item.get_text(strip=True))
        all_rows_list.append(current_row)

    xg_df = pd.DataFrame(all_rows_list, columns = headers_list)    
    
    return xg_df

In [5]:
def xG_Team_Scraper(soup):
    # Returns a dataframe containing a particlar team's players' xg data
   
    headers = soup.find('div', attrs={'class':'players margin-top jTable'}).find('table').find_all('th',attrs={'class':'sort'})
    header_list = []

    for header in headers:
        header_list.append(header.get_text(strip = True))

    header_list = ['№', 'Player','Pos','Apps', 'Min', 'G', 'A', 'Sh90', 'KP90', 'xG', 'xA', 'xG90', 'xA90']
    
    body = soup.find('div', attrs={'class':'players margin-top jTable'}).find('table').find('tbody')
    all_rows_list = []

    for tr in body.find_all('tr'):
        row = tr.find_all('td')
        current_row = []
        for item in row:
            current_row.append(item.get_text(strip = True))
        all_rows_list.append(current_row)
    
    team_xg = pd.DataFrame(all_rows_list, columns = header_list)
    
   
    return team_xg

In [6]:
def get_all_player_xg(teams):
    #Returns a dataframe of all player stats in EPL
    
    df = pd.DataFrame()
    
    for team in teams:
        team_soup = get_team_soup(team)
        team_df = xG_Team_Scraper(team_soup)

        team_df.insert(1,column = 'Team', value = team)

        df = pd.concat([df, team_df], ignore_index=True)

        # Use timer to avoid requesting data from understat too frequently
        time.sleep(3)
            
        print(f"{team}: Done")
    
    return df

In [7]:
def get_json(file_path):
    # Get json file from fanatsy.premierleague
    r = requests.get('https://fantasy.premierleague.com/drf/bootstrap')
    jsonResponse = r.json()
    with open(file_path, 'w') as outfile:
        json.dump(jsonResponse, outfile)

In [8]:
def get_fixtures_EPL(epl_players):
    # Updates the EPL player dataframe with upcoming fixture information
    
    get_json(r"C:\Users\TinyPoots\OneDrive\Python\FPL\fpl.json")
    
    with open(r"C:\Users\TinyPoots\OneDrive\Python\FPL\fpl.json") as json_data:
        d = json.load(json_data)
    
    # normalize json in to flat table
    df = json_normalize(d['teams'])
    id_df = df[['id','name']]
    
    # Several of the names on from Fantasy.premierleague are different from the name used on Understat
    id_df['name'].replace({'Wolves':'Wolverhampton Wanderers','Newcastle':'Newcastle United','Man City':'Manchester City', 
                                   'Man Utd':'Manchester United', 'Spurs':'Tottenham'}, inplace = True)
    
    team_id = dict(zip(list(id_df['id']),list(id_df['name'])))
    
    df = json_normalize(d['next_event_fixtures'])
    fixtures_df = df[['event','team_h','team_a']]
    
    # Change team names to the ones used in Understat data
    fixtures_df['team_a'] = fixtures_df['team_a'].map(team_id)
    fixtures_df['team_h'] = fixtures_df['team_h'].map(team_id)
    
    home_team = list(fixtures_df['team_h'])
    away_team = list(fixtures_df['team_a'])
    fixture_dict = dict(zip(home_team, away_team))
    fixture_dict.update(dict(zip(away_team, home_team)))
    
    # Add upcoming fixture data to the players dataframe
    epl_players['Next Opp'] = epl_players['Team'].map(fixture_dict)
    epl_players['Home'] = epl_players['Next Opp'].apply(lambda x: True if (x in away_team) else False)
    
    return epl_players

In [9]:
# Get xG data for every team in the EPL
epl_df = xG_EPL_Scraper(get_EPL_soup())
# Save for later use
epl_df.to_csv('xg_EPL.csv')

In [10]:
# Get xG data for every player in the EPL
team_names = list(epl_df['Team'])
player_df = get_all_player_xg(team_names)

Liverpool: Done
Manchester City: Done
Tottenham: Done
Arsenal: Done
Manchester United: Done
Chelsea: Done
Wolverhampton Wanderers: Done
Watford: Done
West Ham: Done
Leicester: Done
Everton: Done
Bournemouth: Done
Newcastle United: Done
Crystal Palace: Done
Brighton: Done
Southampton: Done
Burnley: Done
Cardiff: Done
Fulham: Done
Huddersfield: Done


In [11]:
# Add upcoming fixture information
players_df_fix = get_fixtures_EPL(player_df)

C:\Users\TinyPoots\Anaconda3\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\TinyPoots\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\TinyPoots\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

In [12]:
players_df_fix.to_csv('xg_Players.csv')

In [16]:
players_df_fix.sort_values('G', ascending = False)

,№,Team,Player,Pos,Apps,Min,G,A,Sh90,KP90,xG,xA,xG90,xA90,Next Opp,Home
100,3,Manchester United,Anthony Martial,F M,21,1309,9,2,2.06,2.06,5.49-3.51,3.00+1.00,0.38,0.21,Wolverhampton Wanderers,True
101,4,Manchester United,Marcus Rashford,F M,26,1820,9,6,3.21,1.34,9.60+0.60,4.30-1.70,0.47,0.21,Wolverhampton Wanderers,True
317,1,Crystal Palace,Luka Milivojevic,M,30,2700,9,1,1.60,1.40,8.60-0.40,2.31+1.31,0.29,0.08,Tottenham,True
392,1,Burnley,Ashley Barnes,F,30,1766,9,2,2.70,0.97,11.50+2.50,3.14+1.14,0.59,0.16,Wolverhampton Wanderers,True
293,1,Newcastle United,Salomón Rondón,F,25,1986,9,5,2.95,1.04,8.27-0.73,2.76-2.24,0.37,0.13,Arsenal,False
169,1,Watford,Troy Deeney,F,27,2188,8,5,2.14,1.40,8.54+0.54,4.38-0.62,0.35,0.18,Manchester United,True
24,3,Manchester City,Leroy Sané,F,25,1530,8,9,2.59,2.00,6.28-1.72,6.91-2.09,0.37,0.41,Fulham,True
194,1,West Ham,Felipe Anderson,F M,31,2623,8,4,1.51,1.89,4.37-3.63,5.61+1.61,0.15,0.19,Everton,True
127,2,Chelsea,Pedro,F,25,1485,8,1,2.79,1.27,5.34-2.66,2.55+1.55,0.32,0.15,Cardiff,True
318,2,Crystal Palace,Wilfried Zaha,F M,26,2328,8,2,2.28,1.51,4.90-3.10,3.21+1.21,0.19,0.12,Tottenham,True
